Que3.-

The model's main purpose is to predict the 10-year risk of Coronary Heart Disease (CHD). CHD is a 
disease of the blood vessels supplying the heart. Heart disease has been the leading cause of death worldwide
since 1921. In 2008, 7.3 million people died from CHD. 
The classification goal is to predict whether the patient has 10-year risk of future coronary heart disease 
(CHD).The dataset provides the patients’ information. It includes over 4,000 records and 15 attributes.
Variables
Sex: male or female(0=female, 1=male)
Age: Age of the patient
education: Some high school (1), high school/GED (2), some college/vocational school (3), college (4)
Current Smoker: whether or not the patient is a current smoker (0=No, 1=Yes)
Cigs Per Day: the number of cigarettes that the person smoked on average in one day
BP Meds: whether or not the patient was on blood pressure medication 
Prevalent Stroke: whether or not the patient had previously had a stroke (0=No, 1=Yes)
Prevalent Hyp: whether or not the patient was hypertensive (0=No, 1=Yes)
Diabetes: whether or not the patient had diabetes (0=No, 1=Yes)
Tot Chol: total cholesterol level
Sys BP: systolic blood pressure 
Dia BP: diastolic blood pressure
BMI: Body Mass Index
Heart Rate: heart rate
Glucose: glucose level

Prepare model by using logistic regression.

Also calculate Accuracy by confusion matrix, Calculate accuracy score, precision score, recall score, f1 score, 
roc auc score. Also Visualize the result. Also select best parameters using gridsearchCV. 

Datasets : framingham.csv


#### import packages

In [25]:
import warnings
warnings.filterwarnings(action="ignore")

In [26]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load the data

In [27]:
df = pd.read_csv("framingham.csv")
df.head(3)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0


In [28]:
df.drop(['education'], axis=1, inplace=True)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4240 entries, 0 to 4239
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4240 non-null   int64  
 1   age              4240 non-null   int64  
 2   currentSmoker    4240 non-null   int64  
 3   cigsPerDay       4211 non-null   float64
 4   BPMeds           4187 non-null   float64
 5   prevalentStroke  4240 non-null   int64  
 6   prevalentHyp     4240 non-null   int64  
 7   diabetes         4240 non-null   int64  
 8   totChol          4190 non-null   float64
 9   sysBP            4240 non-null   float64
 10  diaBP            4240 non-null   float64
 11  BMI              4221 non-null   float64
 12  heartRate        4239 non-null   float64
 13  glucose          3852 non-null   float64
 14  TenYearCHD       4240 non-null   int64  
dtypes: float64(8), int64(7)
memory usage: 497.0 KB


In [30]:
df.isna().sum()

male                 0
age                  0
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [31]:
df['cigsPerDay'] = df['cigsPerDay'].fillna(df['cigsPerDay'].mean())

In [32]:
df['BPMeds'] = df['BPMeds'].fillna(df['BPMeds'].mean())

In [33]:
df['totChol'] = df['totChol'].fillna(df['totChol'].mean())

In [34]:
df['BMI'] = df['BMI'].fillna(df['BMI'].mean())

In [35]:
df['heartRate'] = df['heartRate'].fillna(df['heartRate'].mean())

In [36]:
df['glucose'] = df['glucose'].fillna(df['glucose'].mean())

In [37]:
df.isna().sum()

male               0
age                0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

In [38]:
df.corr()

,male,age,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
male,1.000000,-0.029014,0.197026,0.316023,-0.052203,-0.004550,0.005853,0.015693,-0.070064,-0.035879,0.058199,0.081705,-0.116913,0.005718,0.088374
age,-0.029014,1.000000,-0.213662,-0.192534,0.122036,0.057679,0.306799,0.101314,0.260691,0.394053,0.205586,0.135578,-0.012839,0.116951,0.225408
currentSmoker,0.197026,-0.213662,1.000000,0.767055,-0.048621,-0.032980,-0.103710,-0.044285,-0.046211,-0.130281,-0.107933,-0.167483,0.062678,-0.054062,0.019448
cigsPerDay,0.316023,-0.192534,0.767055,1.000000,-0.045847,-0.032711,-0.066444,-0.037086,-0.026182,-0.088523,-0.056473,-0.092888,0.075257,-0.056020,0.057646
BPMeds,-0.052203,0.122036,-0.048621,-0.045847,1.000000,0.115008,0.259125,0.051584,0.078973,0.252023,0.192387,0.099586,0.015172,0.048925,0.086805
prevalentStroke,-0.004550,0.057679,-0.032980,-0.032711,0.115008,1.000000,0.074791,0.006955,0.000105,0.057000,0.045153,0.024856,-0.017674,0.018065,0.061823
prevalentHyp,0.005853,0.306799,-0.103710,-0.066444,0.259125,0.074791,1.000000,0.077752,0.162683,0.696656,0.615840,0.300599,0.146777,0.082757,0.177458
diabetes,0.015693,0.101314,-0.044285,-0.037086,0.051584,0.006955,0.077752,1.000000,0.040161,0.111265,0.050260,0.086282,0.048986,0.605709,0.097344
totChol,-0.070064,0.260691,-0.046211,-0.026182,0.078973,0.000105,0.162683,0.040161,1.000000,0.207436,0.163423,0.115013,0.090678,0.044710,0.081807
sysBP,-0.035879,0.394053,-0.130281,-0.088523,0.252023,0.057000,0.696656,0.111265,0.207436,1.000000,0.783952,0.325172,0.182084,0.134561,0.216374


In [39]:
df['TenYearCHD'].value_counts()

TenYearCHD
0    3596
1     644
Name: count, dtype: int64

In [43]:
x = df.drop(['male', 'currentSmoker', 'cigsPerDay', 'TenYearCHD'], axis=1)
y=df['TenYearCHD']

In [44]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_sampled,y_sampled = smote.fit_resample(x,y)

In [45]:
x_sampled.value_counts()

age  BPMeds  prevalentStroke  prevalentHyp  diabetes  totChol  sysBP  diaBP  BMI    heartRate  glucose  
70   1.0     0                1             0         231.0    136.0  84.0   31.78  60.0       95.000000    1
32   0.0     0                0             0         242.0    111.0  70.0   29.84  80.0       88.000000    1
33   0.0     0                0             0         158.0    108.0  67.0   19.84  86.0       69.000000    1
                                                      165.0    136.0  75.0   24.95  88.0       90.000000    1
                                                      199.0    116.0  81.0   21.61  75.0       93.000000    1
                                                                                                           ..
34   0.0     0                0             0         184.0    118.0  77.0   25.56  86.0       81.963655    1
                                                      180.0    111.0  56.0   21.51  91.0       78.000000    1
               

In [46]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_sampled,y_sampled,train_size=0.7,random_state=123456)

In [61]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C= np.float64(0.8), penalty='l1', solver= 'liblinear')
model.fit(x_train,y_train)

LogisticRegression(C=np.float64(0.8), penalty='l1', solver='liblinear')

In [62]:
y_pred = model.predict(x_test)
y_true = y_test

###### confusion matrix, Calculate accuracy score, precision score, recall score, f1 score, roc auc score

In [63]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
cm = confusion_matrix(y_pred,y_true)
accuracy= accuracy_score(y_pred,y_true)
precision = precision_score(y_pred,y_true)
recall = recall_score(y_pred,y_true)
f1 = f1_score(y_pred,y_true)
roc = roc_auc_score(y_pred,y_true)
print(f"confusion matrix = {cm}")
print(f" precision = {precision}")
print(f" recall = {recall}")
print(f" f1 = {f1}")
print(f" roc score = {roc}")

confusion matrix = [[719 330]
 [377 732]]
 precision = 0.6892655367231638
 recall = 0.6600541027953111
 f1 = 0.6743436204514048
 roc score = 0.6727343917217737


penalty{‘l1’, ‘l2’, ‘elasticnet’, None}

solver{‘lbfgs’, ‘liblinear’, ‘newton-cg’, ‘newton-cholesky’, ‘sag’, ‘saga’},

In [64]:
from sklearn.model_selection import GridSearchCV
hyper_parameters = {
    "penalty":['l1','l2','elasticnet','None'],
    "C":np.arange(10)*0.1,
    "solver":['lbfgs', 'liblinear', 'newton-cg', 'newton-chlesky','sag','saga']
}

grid_search_cv = GridSearchCV(estimator=model, param_grid=hyper_parameters)

In [56]:
grid_search_cv

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                         'penalty': ['l1', 'l2', 'elasticnet', 'None'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-chlesky', 'sag', 'saga']})

In [57]:
grid_search_cv.fit(x_train,y_train)

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                         'penalty': ['l1', 'l2', 'elasticnet', 'None'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-chlesky', 'sag', 'saga']})

In [58]:
grid_search_cv.best_score_

np.float64(0.6628909758924112)

In [59]:
grid_search_cv.best_params_

{'C': np.float64(0.8), 'penalty': 'l1', 'solver': 'liblinear'}